In [1]:
import datetime
%load_ext autoreload
%autoreload 2

In [9]:
import pandas as pd
from datetime import datetime

In [3]:
full_demand = pd.read_csv('./raw/household_power_consumption.txt', sep=';', low_memory=False)

In [4]:
full_demand

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16/12/2006,17:24:00,4.216,0.418,234.840,18.400,0.000,1.000,17.0
1,16/12/2006,17:25:00,5.360,0.436,233.630,23.000,0.000,1.000,16.0
2,16/12/2006,17:26:00,5.374,0.498,233.290,23.000,0.000,2.000,17.0
3,16/12/2006,17:27:00,5.388,0.502,233.740,23.000,0.000,1.000,17.0
4,16/12/2006,17:28:00,3.666,0.528,235.680,15.800,0.000,1.000,17.0
...,...,...,...,...,...,...,...,...,...
2075254,26/11/2010,20:58:00,0.946,0.000,240.430,4.000,0.000,0.000,0.0
2075255,26/11/2010,20:59:00,0.944,0.000,240.000,4.000,0.000,0.000,0.0
2075256,26/11/2010,21:00:00,0.938,0.000,239.820,3.800,0.000,0.000,0.0
2075257,26/11/2010,21:01:00,0.934,0.000,239.700,3.800,0.000,0.000,0.0


In [17]:
demand = full_demand[['Date', 'Time', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']].copy()
demand.loc[:,'Datetime'] = pd.to_datetime(demand.loc[:, 'Date'] + ' ' + demand.loc[:,'Time'], dayfirst=True)+ pd.DateOffset(years = 14)
demand.set_index('Datetime', inplace=True)
demand = demand.loc[:, ['Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']]

In [18]:
demand

,Sub_metering_1,Sub_metering_2,Sub_metering_3
Datetime,,,
2020-12-16 17:24:00,0.000,1.000,17.0
2020-12-16 17:25:00,0.000,1.000,16.0
2020-12-16 17:26:00,0.000,2.000,17.0
2020-12-16 17:27:00,0.000,1.000,17.0
2020-12-16 17:28:00,0.000,1.000,17.0
...,...,...,...
2024-11-26 20:58:00,0.000,0.000,0.0
2024-11-26 20:59:00,0.000,0.000,0.0
2024-11-26 21:00:00,0.000,0.000,0.0


In [26]:
demand_episodes = demand.groupby(pd.Grouper(freq='W'))

In [27]:
demand_episodes = pd.concat([group.fillna(0) for name, group in demand_episodes if len(group.dropna()) >= 10000 and group.index[0] >= datetime(2021, 1, 1) and group.index[-1] <= datetime(2022, 12, 31)]).groupby(pd.Grouper(freq='W'))

In [28]:
demand_episodes.describe()

Sub_metering_3                                               
                    count      mean       std  min  25%  50%   75%   max
Datetime                                                                
2021-01-10        10080.0  7.473710  8.628266  0.0  0.0  0.0  17.0  19.0
2021-01-17        10080.0  8.215972  8.739456  0.0  0.0  0.0  18.0  19.0
2021-01-24        10080.0  7.129762  8.584038  0.0  0.0  0.0  17.0  19.0
2021-01-31        10080.0  8.169742  8.774012  0.0  0.0  0.0  18.0  20.0
2021-02-07        10080.0  8.049702  8.719949  0.0  0.0  0.0  17.0  20.0
...                   ...       ...       ...  ...  ...  ...   ...   ...
2022-11-27        10080.0  7.468155  8.677335  0.0  0.0  0.0  17.0  19.0
2022-12-04        10080.0  7.594742  8.821079  0.0  0.0  0.0  18.0  20.0
2022-12-11        10080.0  7.692163  8.877297  0.0  0.0  0.0  18.0  21.0
2022-12-18        10080.0  6.939286  8.694680  0.0  0.0  0.0  18.0  20.0
2022-12-25        10080.0  7.117659  8.711846  0.0  0.0  0.0  18.0  20.0

[103 rows x 8 columns]

In [29]:
with pd.HDFStore('./minutely/demand.h5') as store:
    # Save each DataFrame with a key
    for i, (name, group) in enumerate(demand_episodes):
        store[f'eps_{i}'] = group